In [ ]:
from operator import index
from dataDownloader  import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
from db_stock import DB_Stock
from portfolio import Portfolio
from assetAllocation import AssetAllocation
from matplotlib.dates import relativedelta
from datetime import datetime, timedelta
from commonHelper import EDateType
from scipy.stats.mstats import winsorize

import datetime
import dataDownloader
import yfinance as yf;
import pandas as pd
import commonHelper
import numpy as np
import os
import warnings
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

start_date = '2014-12-01'
end_date = '2025-08-01'



symbols = []
with DB_FinancialStatement() as fs:
    symbols = fs.get_symbol_list_with_filter(2022)
    symbols = symbols[:1]
    # symbols = ['DTST', 'DXLG', 'AEHR', 'AMTX', 'BGI', 'EP', 'PMTS', 'DTST', 'MPU', 'DXLG', 'GWAV', 'ODV']

df_symbols = AssetAllocation.get_stock_data_with_ma(
    symbols=symbols, 
    start_date=start_date, 
    end_date=end_date, 
    mas=[10], 
    type='ma_month',
    use_db_stock=True)

df_symbols = AssetAllocation.filter_close_last_month(df_symbols)

df_hedge = AssetAllocation.get_stock_data_with_ma(
    symbols=['SPY', '^VIX'], 
    start_date=start_date, 
    end_date=end_date, 
    mas=[10], 
    type='ma_month',
    use_db_stock=False)


df_hedge = AssetAllocation.filter_close_last_month(df_hedge)


# 3-12 상승, 2년 zscore사용
def strategy_power_momentum(df_symbols:dict, df_hedge:dict, m:int=3, window:int=24):
    # m = 3
    col_m_return = f"R_{m}_12M" # 중기 모멘텀
    col_m_reversal = f"R_{m}M"  # 단기 모멘텀
    col_z_score = f"R_{m}M_Z"   # Z스코어
    col_over_return = "Can Over Return"

    df_symbols = df_symbols.copy()

    for key, df in df_symbols.items():
        df = df.copy()

        # 필요 없는 컬럼 제거
        df = df[[col for col in df.columns if col not in ['Dividends', 'Adj Close', 'Condition', 'Close']]]

        # 수익률 컬럼 (월별)
        df["Monthly_Return"] = df["Result Close"].pct_change(fill_method=None)

        # R_2_12M, R_2M 계산
        df[col_m_return] = (df["Result Close"].shift(m) / df["Result Close"].shift(12)) - 1
        df[col_m_reversal] = (df["Result Close"] / df["Result Close"].shift(m)) - 1    

        # window = 24  # 36개월 (3년)

        # rolling 평균과 표준편차
        rolling_mean = df[col_m_reversal].rolling(window=window, min_periods=window).mean()
        rolling_std = df[col_m_reversal].rolling(window=window, min_periods=window).std()

        # 각 row별 "과거 36개월 기준 Z-score"
        df[col_z_score] = (df[col_m_reversal] - rolling_mean) / rolling_std

        # 상승횟수 vs 하락횟수 (3~12개월 전 구간)
        def count_up_down(idx):
            # 3~12개월 전 수익률 가져오기
            window = df.loc[idx-12:idx-m, "Monthly_Return"]
            if window.empty:
                return np.nan
            ups = (window > 0).sum()
            downs = (window < 0).sum()
            return 1 if ups > downs else 0

        # 새로운 컬럼 추가
        df[col_over_return] = [count_up_down(i) for i in range(len(df))]
        df = df.dropna(subset=[col_m_return, col_z_score])
        df_symbols[key] = df

    

    df = AssetAllocation.merge_to_dfs(df_symbols, [col_m_reversal, 'Monthly_Return', 'MMA_10'])    
    df = df.loc[df.groupby(df["Date"].dt.to_period("M"))["Date"].idxmax()]
    df = df.sort_values("Date").reset_index(drop=True)

    # 도중에 상폐된 심볼에 대한 제거
    # symbol_counts = df['Symbol'].astype(str).value_counts()
    # main_pattern = symbol_counts.index[0]
    # df = df[df['Symbol'].astype(str) == main_pattern].reset_index()
    # df = df.drop(columns=["index"])

    init_balance = 10000

    df['End Balance'] = None        # 지난 달 Restart Asset의 결과        
    df['Restart Asset'] = None      # 현 월 마지막에 리벨런싱 처리할 대상
    df['Restart Balance'] = None    # 리벨런싱의 자산 배분
    df['Cash'] = 0                  # 현금 보유 상황
    df['Balance'] = None            # 토탈 비용
    df['Enter Ratio'] = 0.0
    df['VIX'] = 0.0

    def get_restart_assets(symbols, df):
        m_return = dict(zip(symbols, df.at[i, col_m_return])) # 중기모멘텀
        zscore = dict(zip(symbols, df.at[i, col_z_score])) # z스코어
        over_return = dict(zip(symbols, df.at[i, col_over_return]))

        symbols = [sym for sym in symbols
                if -2<=zscore[sym]<=2 and  # Z스코어는 상승/하락 모두 포함하는게 좋음.
                    over_return[sym] == 1 and 
                    m_return[sym] > 0]
        
        symbols = sorted(symbols, key=lambda sym: m_return[sym], reverse=True)
        symbols = symbols[:20]
        return symbols

    # 이전달에 투자한 자산의 결과 리스트 반환
    def get_end_balance(prev_assets, s_symbol, s_close, e_symbol, e_close):
        end_balance = []
        if prev_assets:
            start_close_dict = dict(zip(s_symbol, s_close))
            end_close_dict = dict(zip(e_symbol, e_close))

            try:
                change_ratio = [
                    end_close_dict[sym] / start_close_dict[sym]
                    if start_close_dict[sym] != 0 or sym in end_close_dict 
                        else 0
                    for sym in prev_assets
                ]
            except Exception as e:
                raise Exception((f"애러발생! : {prev_assets}, {e}"))

            start_balance = df.at[i-1, 'Restart Balance']
            end_balance = [round(x*y,2) for x,y in zip(start_balance, change_ratio)]
            df.at[i, 'End Balance'] = end_balance

        return end_balance

    # 한번에 자산 배분할때 얼마만큼 배분할 수 있는지
    def get_one_invest_balance(enter_ratio:float, balance:int, count:int):

        balance = enter_ratio*balance
        one_invest_balance = min(balance//10, balance//count)

        return one_invest_balance
    

    def get_hedge_value(df_hedge, date):   
        enter_ratio = 0
        vix_value = 0

        if '^VIX' in df_hedge:
            df_vix = df_hedge['^VIX']
            mask = df_vix['Date'] == date
            if mask.any():
                vix_value = df_vix.loc[mask, 'Result Close'].iloc[0]

        if vix_value < 15:
            enter_ratio = 1
        elif  15<= vix_value <20:
            enter_ratio = 0.75
        elif 20<= vix_value<30:
            enter_ratio = 0.5
        elif 30<= vix_value:
            enter_ratio = 0.25
        else:
            enter_ratio = 0

        if 'SPY' in df_hedge: 
            df_spy = df_hedge['SPY']
            is_over_spy = (df_spy.loc[df_spy['Date'] == date, 'Result Close'] > 
                            df_spy.loc[df_spy['Date'] == date, 'MMA_10']).any()
            if not is_over_spy:
                enter_ratio = 0

        return (enter_ratio, vix_value)

        

    for i in range(len(df)):
        date = df.at[i, 'Date']
        
        curr_assets = get_restart_assets(df.at[i, 'Symbol'], df)

        df.at[i,'Restart Asset'] = curr_assets

        enter_ratio, vix_value = get_hedge_value(df_hedge, date)
        df.at[i, 'VIX'] = int(vix_value)
        df.at[i, 'Enter Ratio'] = enter_ratio
        
        if i == 0:
            if not curr_assets:
                df.at[0, 'Cash'] = init_balance
            else:
                curr_assets_len = len(curr_assets)
                one_invest_balance = get_one_invest_balance(enter_ratio, init_balance, curr_assets_len)

                df.at[0,'Restart Balance'] = [one_invest_balance]*curr_assets_len
                df.at[0,'Cash'] = init_balance - (one_invest_balance*curr_assets_len)
                
            df.at[0, 'Balance'] = init_balance

        else:
            prev_assets = df.at[i-1, 'Restart Asset']

            end_balance = get_end_balance(prev_assets, 
                                        df.at[i-1,'Symbol'], 
                                        df.at[i-1,'Result Close'],
                                        df.at[i, 'Symbol'],
                                        df.at[i, 'Result Close'])
            now_total_balance = (
                df.at[i-1, 'Cash']
                if not end_balance
                    else  df.at[i-1, 'Cash'] + sum(end_balance)
            )

            df.at[i, 'Balance'] = now_total_balance

            if not curr_assets:
                df.at[i, 'Cash'] = now_total_balance       
            else:
                curr_assets_len = len(curr_assets)
                one_invest_balance = get_one_invest_balance(enter_ratio, now_total_balance, curr_assets_len)

                restart_balance = [one_invest_balance] * curr_assets_len
                cash = int(now_total_balance - (one_invest_balance * curr_assets_len))

                df.at[i, 'Restart Balance'] = restart_balance
                df.at[i, 'Cash'] = cash
    
    return df


# allocations = [
#     {'3R 24 VIX' : strategy_power_momentum(df_symbols, df_hedge, 3, 24)},
#     {'2R 24 VIX' : strategy_power_momentum(df_symbols, df_hedge, 2, 24)},
#     {'1R 24 VIX' : strategy_power_momentum(df_symbols, df_hedge, 1, 24)},
#     {'3R 36 VIX' : strategy_power_momentum(df_symbols, df_hedge, 3, 36)},
#     {'2R 36 VIX' : strategy_power_momentum(df_symbols, df_hedge, 2, 36)},
#     {'1R 36 VIX' : strategy_power_momentum(df_symbols, df_hedge, 1, 36)}, 
# ]

portfolio_name = [
    '1R 36',
    '1R 36 SPY',
    '1R 36 VIX',
]

allocations = []
for name in portfolio_name:
    key_value = {}
    key_value[name] = pd.read_csv(f"stocks/power_momentum_{name}.csv")
    allocations.append(key_value)


for pair in allocations:
    for key, value in pair.items():
        value.to_csv(f'stocks/power_momentum_{key}.csv', index = False)

Portfolio.show_portfolio(allocations)


# df = pd.read_csv("stocks/power_momentum_1R 36.csv")
# display(df)


# df = strategy_power_momentum(df_symbols, df_hedge, 3, 24)
# display(df)
# df = strategy_power_momentum(df_symbols, df_spy, 2, 24)
# display(df)


[get_dividend] 배당 데이터 없음 : ['^VIX']


,Date,Symbol,Result Close,R_1_12M,R_1M_Z,Can Over Return,End Balance,Restart Asset,Restart Balance,Cash,Balance,Over SPY
0,2017-12-29,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[66.97, 53.87, 52.03, 3.4, 37.82, 24.47, 99.69...","[0.52, 0.48, 0.08, -0.12, 0.86, 0.1, -0.4, 0.4...","[-0.78, 2.16, 0.3, -0.67, -0.82, -0.14, -0.04,...","[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",NaN,"['TSSI', 'ZYXI', 'GRVY', 'XOMA', 'SGMO', 'DVAX...","[500, 500, 500, 500, 500, 500, 500, 500, 500, ...",0,10000.00,True
1,2018-01-31,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[73.43, 52.02, 54.32, 3.75, 32.39, 24.27, 116....","[0.37, 0.48, 0.18, -0.09, 0.23, 0.08, -0.39, 0...","[1.24, -0.4, 0.41, 1.24, -0.88, -0.38, 1.93, -...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","[471.15, 762.98, 366.77, 462.08, 635.67, 430.4...","['TSSI', 'ZYXI', 'XOMA', 'MDGL', 'ESPR', 'GRVY...","[np.float64(679.0), np.float64(679.0), np.floa...",6,13586.26,True
2,2018-02-28,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[68.59, 44.97, 54.25, 3.45, 27.93, 24.5, 114.2...","[0.43, 0.5, 0.17, -0.03, -0.29, 0.08, -0.25, 0...","[-1.34, -1.14, -0.1, -0.95, -0.81, -0.09, -0.2...","[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[np.float64(651.29), np.float64(608.17), np.fl...","['ZYXI', 'TSSI', 'MDGL', 'XOMA', 'NKTR', 'ACB'...","[np.float64(676.0), np.float64(676.0), np.floa...",12,13532.68,True
3,2018-03-29,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[66.9, 44.96, 51.96, 3.3, 25.06, 26.0, 118.55,...","[0.3, 0.31, 0.28, -0.14, -0.5, 0.04, -0.23, 0....","[-0.65, -0.14, -0.51, -0.49, -0.62, 0.73, 0.43...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[np.float64(527.11), np.float64(848.6), np.flo...","['QUBT', 'ZYXI', 'MDGL', 'CCLD', 'EGAN', 'SGMO...","[np.float64(660.0), np.float64(660.0), np.floa...",1,13201.48,True
4,2018-04-30,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[65.74, 51.2, 42.93, 3.25, 31.96, 22.67, 114.4...","[0.22, 0.33, 0.22, -0.16, -0.49, 0.06, -0.17, ...","[-0.53, 0.9, -1.86, -0.16, 0.98, -2.06, -0.35,...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[np.float64(308.0), np.float64(623.57), np.flo...","['QUBT', 'ZYXI', 'CDNA', 'MDGL', 'VERB', 'NKTR...","[np.float64(626.0), np.float64(626.0), np.floa...",8,12528.09,True
5,2018-05-31,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[61.92, 48.07, 43.54, 2.95, 46.77, 20.33, 128....","[0.09, 0.55, -0.11, -0.1, -0.54, -0.06, -0.14,...","[-1.15, -0.63, 0.1, -1.05, 1.7, -1.59, 1.33, 1...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[np.float64(693.07), np.float64(615.24), np.fl...","['CDNA', 'ZYXI', 'ENPH', 'QURE', 'VERB', 'VRME...","[np.float64(777.0), np.float64(777.0), np.floa...",5,15545.91,True
6,2018-06-29,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[61.84, 46.88, 37.96, 2.7, 44.9, 22.17, 135.7,...","[0.04, 0.47, -0.13, -0.21, -0.24, -0.17, 0.1, ...","[-0.27, -0.37, -1.4, -0.94, -0.39, 1.08, 0.59,...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[np.float64(718.31), np.float64(766.13), np.fl...","['NIXX', 'QUBT', 'MDGL', 'CDNA', 'VKTX', 'EGAN...","[np.float64(734.0), np.float64(734.0), np.floa...",16,14696.77,True
7,2018-07-31,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[66.04, 43.27, 39.54, 2.6, 38.43, 25.17, 141.2...","[0.03, 0.29, -0.25, -0.24, -0.54, -0.02, 0.21,...","[0.87, -0.79, 0.4, -0.36, -0.78, 1.6, 0.46, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[np.float64(760.69), np.float64(1004.42), np.f...","['QUBT', 'MDGL', 'CDNA', 'VKTX', 'EGAN', 'ARWR...","[np.float64(795.0), np.float64(795.0), np.floa...",12,15912.37,True
8,2018-08-31,"['A', 'AA', 'AAL', 'AAME', 'AAOI', 'AAON', 'AA...","[67.54, 44.67, 40.48, 2.75, 41.36, 26.93, 164....","[0.02, -0.01, -0.12, -0.2, -0.35, 0.16, 0.44, ...","[0.07, 0.05, 0.2, 0.72, 0.12, 0.67, 1.64, 2.41...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[np.float64(764.42), np.float64(739.88), np.fl...","['QUBT', 'MDGL', 'VKTX', 'TBCH', 'EGAN